In [1]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler, StandardScaler
from sklearn.metrics import classification_report
from sklearn.svm import OneClassSVM
from sklearn.pipeline import Pipeline
import os
import glob
import csv
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn import preprocessing

In [2]:
os.chdir("D:/Data")
csv_data_path = 'D:/Data/'

In [5]:
#Create a dataframe and the csv file

files = ['02-14-2018.csv',
        '02-15-2018.csv',
        '02-16-2018.csv',
        '02-20-2018.csv',
        '02-21-2018.csv',
        '02-22-2018.csv',
        '02-23-2018.csv',
        '02-28-2018.csv',
        '03-01-2018.csv',
        '03-02-2018.csv']


In [6]:
def fusionfiles(files):
    dfs = []
    for i in range (10):
        df = pd.read_csv(files[i],nrows=10000)
        dfs.append(df)
    return dfs

In [7]:
dfs = []
dfs = fusionfiles(files)

C:\Users\hp\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3338: DtypeWarning: Columns (0,1,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78) have mixed types.Specify dtype option on import or set low_memory=False.
  if (await self.run_code(code, result,  async_=asy)):


In [8]:
# Concatenation de toute les données dans un seul DataFrame
data_df = pd.concat(dfs)
data_df = data_df.drop([ 'Flow ID', 'Src IP', 'Dst IP', 'Src Port','Timestamp'], axis=1)
data_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 100000 entries, 0 to 9999
Data columns (total 79 columns):
 #   Column             Non-Null Count   Dtype 
---  ------             --------------   ----- 
 0   Dst Port           100000 non-null  object
 1   Protocol           100000 non-null  object
 2   Flow Duration      100000 non-null  object
 3   Tot Fwd Pkts       100000 non-null  object
 4   Tot Bwd Pkts       100000 non-null  object
 5   TotLen Fwd Pkts    100000 non-null  object
 6   TotLen Bwd Pkts    100000 non-null  object
 7   Fwd Pkt Len Max    100000 non-null  object
 8   Fwd Pkt Len Min    100000 non-null  object
 9   Fwd Pkt Len Mean   100000 non-null  object
 10  Fwd Pkt Len Std    100000 non-null  object
 11  Bwd Pkt Len Max    100000 non-null  object
 12  Bwd Pkt Len Min    100000 non-null  object
 13  Bwd Pkt Len Mean   100000 non-null  object
 14  Bwd Pkt Len Std    100000 non-null  object
 15  Flow Byts/s        99648 non-null   object
 16  Flow Pkts/s        100

In [16]:
data_df.shape

(100000, 79)

In [10]:
data_df['Label'].value_counts()

Benign                      43083
DDoS attacks-LOIC-HTTP       9934
DoS attacks-GoldenEye        9931
FTP-BruteForce               9904
DoS attacks-SlowHTTPTest     9899
Bot                          8520
DDOS attack-HOIC             6070
DDOS attack-LOIC-UDP         1730
Brute Force -Web              611
Brute Force -XSS              230
SQL Injection                  87
Label                           1
Name: Label, dtype: int64

In [14]:
#suppression des champs infinity et inf
data_df = data_df.replace('Infinity', 'inf')
#transformation des données en numérique
for column in data_df.columns:
    if(column!="Label"):
        data_df[column] = pd.to_numeric(data_df[column], errors='coerce')
data_df.dtypes

Dst Port         float64
Protocol         float64
Flow Duration    float64
Tot Fwd Pkts     float64
Tot Bwd Pkts     float64
                  ...   
Idle Mean        float64
Idle Std         float64
Idle Max         float64
Idle Min         float64
Label              int32
Length: 79, dtype: object

In [15]:
#remplacement des données vide par la moyenne de la variable dont lequel elles sont présente
data_df.dropna()
data_df.fillna(data_df.mean(), inplace=True)

In [ ]:
#création du fichier avec les variables qu'il faut et leur format 
data_df.to_csv (r'dt_2018_dataframe.csv', index = False, header=True)

In [17]:
data_df = pd.read_csv('dt_2018_dataframe.csv')

In [18]:
#séparation du dataset en données d'entrainement et de test
train = data_df.groupby('Label').apply(pd.DataFrame.sample, frac=0.8).reset_index(level='Label', drop=True)
test = data_df.drop(train.index)
label = train.Label.copy()

In [19]:
#Encodez les étiquettes cibles des données d'entrainement  avec une valeur 0 pour les comportement normale et 1 pour les attaques
d1 = train.replace('Benign', 0)
d2 = d1.replace('Bot', 1)
d3 = d2.replace('Brute Force -Web', 1)
d4 = d3.replace('Brute Force -XSS', 1)
d5 = d4.replace('DDOS attack-HOIC', 1)
d6 = d5.replace('DDOS attack-LOIC-UDP', 1)
d7 = d6.replace('DDoS attacks-LOIC-HTTP', 1)
d8 = d7.replace('DoS attacks-GoldenEye', 1)
d9 = d8.replace('DoS attacks-SlowHTTPTest', 1)
d10 = d9.replace('FTP-BruteForce', 1)
d11 = d10.replace('SQL Injection', 1)
d12_label = d11.Label.copy()
d12_label.unique()
d12_label.value_counts()

1    45533
0    34466
Name: Label, dtype: int64

In [20]:
#Encodez les étiquettes cibles des données de testes avec une valeur 0 pour les comportement normale et 1 pour les attaques
test_label = test.Label.copy() #ra kanggo
test_label.unique() #ra kanggo
a1_label = test.Label.copy()
a1_label.unique()
a1 = test.replace('Benign', 0)
a2 = a1.replace('Bot', 1)
a3 = a2.replace('Brute Force -Web', 1)
a4 = a3.replace('Brute Force -XSS', 1)
a5 = a4.replace('DDOS attack-HOIC', 1)
a6 = a5.replace('DDOS attack-LOIC-UDP', 1)
a7 = a6.replace('DDoS attacks-LOIC-HTTP', 1)
a8 = a7.replace('DoS attacks-GoldenEye', 1)
a9 = a8.replace('DoS attacks-SlowHTTPTest', 1)
a10 = a9.replace('FTP-BruteForce', 1)
a11 = a10.replace('SQL Injection', 1)
a12_label = a11.Label.copy()
a12_label.unique()
a12_label.value_counts()

1    11383
0     8617
Name: Label, dtype: int64

In [24]:
#création de 3 nouvelle variable à partir de la variable Protocol, car elle à que 3 données distinctes parmis toutes les observation
category_variables = ["Protocol"]

for cv in category_variables:
    d11[cv] = d11[cv].astype("category")
    a11[cv] = a11[cv].astype("category")
    
    print("Length of Categories for {} are {}".format(cv , len(d11[cv].cat.categories)))
    print("Categories for {} are {} \n".format(cv ,d11[cv].cat.categories))

Length of Categories for Protocol are 3
Categories for Protocol are Int64Index([0, 6, 17], dtype='int64') 



In [22]:

dummy_variables_2labels = category_variables
    
class preprocessing:
    train_labels = pd.get_dummies(d11, columns = dummy_variables_2labels, prefix=dummy_variables_2labels)
    test_labels = pd.get_dummies(a11, columns = dummy_variables_2labels, prefix=dummy_variables_2labels)

In [23]:
preprocessing.test_labels.info(verbose=True)

<class 'pandas.core.frame.DataFrame'>
Int64Index: 20000 entries, 0 to 99997
Data columns (total 81 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   Dst Port           20000 non-null  int64  
 1   Flow Duration      20000 non-null  int64  
 2   Tot Fwd Pkts       20000 non-null  int64  
 3   Tot Bwd Pkts       20000 non-null  int64  
 4   TotLen Fwd Pkts    20000 non-null  int64  
 5   TotLen Bwd Pkts    20000 non-null  int64  
 6   Fwd Pkt Len Max    20000 non-null  int64  
 7   Fwd Pkt Len Min    20000 non-null  int64  
 8   Fwd Pkt Len Mean   20000 non-null  float64
 9   Fwd Pkt Len Std    20000 non-null  float64
 10  Bwd Pkt Len Max    20000 non-null  int64  
 11  Bwd Pkt Len Min    20000 non-null  int64  
 12  Bwd Pkt Len Mean   20000 non-null  float64
 13  Bwd Pkt Len Std    20000 non-null  float64
 14  Flow Byts/s        20000 non-null  float64
 15  Flow Pkts/s        20000 non-null  float64
 16  Flow IAT Mean      200

In [ ]:
preprocessing.train_labels.to_csv("D:/Data/preprocessed_train_DT_2018.csv")
preprocessing.test_labels.to_csv("D:/Data/preprocessed_test_DT_2018.csv")